In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
import math
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')

In [2]:
# Lendo as bases de dados
dataset_treino = pd.read_csv("material_trabalho/dataset_treino.csv")
dataset_teste = pd.read_csv("material_trabalho/dataset_teste.csv")
dataset_lojas = pd.read_csv("material_trabalho/lojas.csv")

In [3]:
#Retirando do treino os dias que nao abriram
dataset_treino = dataset_treino[dataset_treino['Open'] != 0]
dataset_treino = dataset_treino[dataset_treino['Sales'] != 0]

In [4]:
# Juntando o conjunto de dados de treino e as lojas
dataset_treino = pd.merge(dataset_treino, dataset_lojas, on = "Store")

# Juntando o conjunto de dados de teste e as lojas
dataset_teste = pd.merge(dataset_teste, dataset_lojas, on = "Store")

dataset_teste.head(5)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,5,2015-07-31,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1116,1,4,2015-07-30,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,2231,1,3,2015-07-29,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,3346,1,2,2015-07-28,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,4461,1,1,2015-07-27,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [5]:
# Mudando as variaveis que estão em categorias de letras para categorias numericas

# StoreType
label_encoder = LabelEncoder().fit(dataset_treino['StoreType'])
dataset_treino['StoreType'] = label_encoder.transform(dataset_treino['StoreType'])

dataset_teste['StoreType'] = label_encoder.transform(dataset_teste['StoreType'])

# Assortment
label_encoder = LabelEncoder().fit(dataset_treino['Assortment'])
dataset_treino['Assortment'] = label_encoder.transform(dataset_treino['Assortment'])

dataset_teste['Assortment'] = label_encoder.transform(dataset_teste['Assortment'])

# StateHoliday
dataset_treino['StateHoliday'] = np.array(dataset_treino['StateHoliday'], dtype = str)
label_encoder = LabelEncoder().fit(dataset_treino['StateHoliday'])
dataset_treino['StateHoliday'] = label_encoder.transform(dataset_treino['StateHoliday'])

dataset_teste['StateHoliday'] = label_encoder.transform(dataset_teste['StateHoliday'])

In [6]:
dataset_treino.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,2014-12-31,2605,327,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,2,2014-12-30,6466,703,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,1,2014-12-29,6463,700,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,6,2014-12-27,6057,684,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,3,2014-12-24,3659,421,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [7]:
#Criando Dia, Mes e ano e removendo colunas que nao serao usadas
dataset_treino['Timestamp'] = pd.to_datetime(dataset_treino['Date'])
dataset_treino['Ano'] = dataset_treino['Timestamp'].apply(lambda data: data.year)
dataset_treino['Mes'] = dataset_treino['Timestamp'].apply(lambda data: data.month)
dataset_treino['Dia'] = dataset_treino['Timestamp'].apply(lambda data: data.day)
dataset_treino['SemanaAno'] = dataset_treino['Timestamp'].apply(lambda data: data.weekofyear)

dataset_teste['Timestamp'] = pd.to_datetime(dataset_teste['Date'])
dataset_teste['Ano'] = dataset_teste['Timestamp'].apply(lambda data: data.year)
dataset_teste['Mes'] = dataset_teste['Timestamp'].apply(lambda data: data.month)
dataset_teste['Dia'] = dataset_teste['Timestamp'].apply(lambda data: data.day)
dataset_teste['SemanaAno'] = dataset_teste['Timestamp'].apply(lambda data: data.weekofyear)

#Removendo colunas que nao serao usadas
dataset_treino = dataset_treino.drop(['Date','Customers','Timestamp'], axis=1)
dataset_teste = dataset_teste.drop(['Date','Timestamp'], axis=1)
dataset_treino.head(5)

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Ano,Mes,Dia,SemanaAno
0,1,3,2605,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2014,12,31,1
1,1,2,6466,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2014,12,30,1
2,1,1,6463,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2014,12,29,1
3,1,6,6057,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2014,12,27,52
4,1,3,3659,1,0,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2014,12,24,52


In [8]:
dataset_treino['PromoInterval'].value_counts()

Jan,Apr,Jul,Oct     183739
Feb,May,Aug,Nov      75304
Mar,Jun,Sept,Dec     62050
Name: PromoInterval, dtype: int64

In [9]:
# Separando os meses de Intervalo de Promocao
dataset_treino[['mes1', 'mes2', 'mes3', 'mes4']] = dataset_treino["PromoInterval"].str.split(",", n = 3, expand = True)

dataset_teste[['mes1', 'mes2', 'mes3', 'mes4']] = dataset_teste["PromoInterval"].str.split(",", n = 3, expand = True)

In [10]:
# Transformando os meses em valores numeros
transf_num = {np.nan: 0, 'Jan': 1, 'Feb':2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
              'Jul': 7, 'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
dataset_treino['mes1'] = dataset_treino['mes1'].map(transf_num)
dataset_treino['mes2'] = dataset_treino['mes2'].map(transf_num)
dataset_treino['mes3'] = dataset_treino['mes3'].map(transf_num)
dataset_treino['mes4'] = dataset_treino['mes4'].map(transf_num)

dataset_teste['mes1'] = dataset_teste['mes1'].map(transf_num)
dataset_teste['mes2'] = dataset_teste['mes2'].map(transf_num)
dataset_teste['mes3'] = dataset_teste['mes3'].map(transf_num)
dataset_teste['mes4'] = dataset_teste['mes4'].map(transf_num)

In [11]:
# Criando a variavel intervalo de promocao
dataset_treino['Intervalo_Promo'] = 0

dataset_teste['Intervalo_Promo'] = 0

# Verificando se a loja está em intervalo de promocao
dataset_treino.Intervalo_Promo[((dataset_treino['Mes'] == dataset_treino['mes1'])\
                                | (dataset_treino['Mes'] == dataset_treino['mes2'])\
                                | (dataset_treino['Mes'] == dataset_treino['mes3']) \
                                | (dataset_treino['Mes'] == dataset_treino['mes4']))] = 1

dataset_teste.Intervalo_Promo[((dataset_teste['Mes'] == dataset_teste['mes1'])\
                                | (dataset_teste['Mes'] == dataset_teste['mes2'])\
                                | (dataset_teste['Mes'] == dataset_teste['mes3']) \
                                | (dataset_teste['Mes'] == dataset_teste['mes4']))] = 1

In [12]:
# Quanto tempo o concorrente abriu em anos
dataset_treino['Concorrente_Open'] = dataset_treino['Ano'] - dataset_treino['CompetitionOpenSinceYear']

dataset_teste['Concorrente_Open'] = dataset_teste['Ano'] - dataset_teste['CompetitionOpenSinceYear']

In [13]:
# Quanto tempo a promo2 abriu em anos
dataset_treino['Promo2_Open'] = dataset_treino['Ano'] - dataset_treino['Promo2SinceYear']

dataset_teste['Promo2_Open'] = dataset_teste['Ano'] - dataset_teste['Promo2SinceYear']

In [14]:
#Preenchendo valores NaN
dataset_treino = dataset_treino.fillna(0)
dataset_teste = dataset_teste.fillna(0)

#Removendo Promo_interval
dataset_treino = dataset_treino.drop(['PromoInterval'], axis=1)
dataset_teste = dataset_teste.drop(['PromoInterval'], axis=1)

In [15]:
dataset_treino

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Mes,Dia,SemanaAno,mes1,mes2,mes3,mes4,Intervalo_Promo,Concorrente_Open,Promo2_Open
0,1,3,2605,1,0,0,1,2,0,1270.0,...,12,31,1,0,0,0,0,0,6.0,0.0
1,1,2,6466,1,0,0,1,2,0,1270.0,...,12,30,1,0,0,0,0,0,6.0,0.0
2,1,1,6463,1,0,0,1,2,0,1270.0,...,12,29,1,0,0,0,0,0,6.0,0.0
3,1,6,6057,1,0,0,1,2,0,1270.0,...,12,27,52,0,0,0,0,0,6.0,0.0
4,1,3,3659,1,0,0,1,2,0,1270.0,...,12,24,52,0,0,0,0,0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648304,1109,1,8089,1,1,0,0,2,0,3490.0,...,1,7,2,1,4,7,10,1,2.0,1.0
648305,1109,6,2897,1,0,0,0,2,0,3490.0,...,1,5,1,1,4,7,10,1,2.0,1.0
648306,1109,5,3596,1,0,0,1,2,0,3490.0,...,1,4,1,1,4,7,10,1,2.0,1.0
648307,1109,4,3654,1,0,0,1,2,0,3490.0,...,1,3,1,1,4,7,10,1,2.0,1.0


In [16]:
y_treino = dataset_treino['Sales']
X_treino = dataset_treino.drop(['Sales'], axis=1)

# Dividindo em dados de treino e validação
X_treino, X_valid, y_treino, y_valid = train_test_split(X_treino, y_treino, test_size = 0.04)


In [17]:

# Melhores parametros
params = {"eta": 0.1,
          "max_depth": 10,
          "silent": 1}

num_boost_round = 100

#Estrutura para dar mais performance
dtrain = xgb.DMatrix(X_treino, np.log1p(y_treino)) # dados de treino
dvalid = xgb.DMatrix(X_valid, np.log1p(y_valid)) # dados de validacao

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

modelo = xgb.train(params, dtrain, num_boost_round, evals = watchlist,
  early_stopping_rounds = 100, verbose_eval = True)



[0]	train-rmse:7.43658	eval-rmse:7.441
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 100 rounds.
[1]	train-rmse:6.6946	eval-rmse:6.69879
[2]	train-rmse:6.02693	eval-rmse:6.0309
[3]	train-rmse:5.42617	eval-rmse:5.42983
[4]	train-rmse:4.88567	eval-rmse:4.88912
[5]	train-rmse:4.39929	eval-rmse:4.40258
[6]	train-rmse:3.96178	eval-rmse:3.96497
[7]	train-rmse:3.56825	eval-rmse:3.57123
[8]	train-rmse:3.21435	eval-rmse:3.21732
[9]	train-rmse:2.896	eval-rmse:2.89877
[10]	train-rmse:2.60985	eval-rmse:2.61253
[11]	train-rmse:2.35253	eval-rmse:2.35506
[12]	train-rmse:2.12117	eval-rmse:2.12368
[13]	train-rmse:1.9134	eval-rmse:1.91572
[14]	train-rmse:1.72632	eval-rmse:1.72854
[15]	train-rmse:1.55887	eval-rmse:1.56099
[16]	train-rmse:1.40817	eval-rmse:1.41019
[17]	train-rmse:1.2734	eval-rmse:1.27538
[18]	train-rmse:1.15205	eval-rmse:1.15396
[19]	train-rmse:1.0438	eval-rmse:1.04566
[20]	train-rmse:0.946645	eval-rmse:

In [18]:
from sklearn.metrics import r2_score

#y_pred = model.predict(X_valid)

# Resultado nos dados de validacao
y_pred = modelo.predict(xgb.DMatrix(X_valid), 
                        ntree_limit = modelo.best_ntree_limit)

score = r2_score(y_valid,np.expm1(y_pred))

print(score)

0.8742053260120889


In [20]:
#Ordenando pelo Id
dataset_teste = dataset_teste.sort_values(by=['Id'], ascending=True)

#Preparando os valores de teste e fazendo a predicao
id_teste = dataset_teste['Id']
X_teste = dataset_teste.drop(['Id'], axis=1)

y_pred = modelo.predict(xgb.DMatrix(X_teste), 
                        ntree_limit = modelo.best_ntree_limit)

#Utilizei o log para treinar, portanto tem que elevar
y_pred = np.expm1(y_pred)

#Colocando valor de vendas igual a 0 quando a loja nao abre
y_pred[dataset_teste['Open'] == 0] = 0

calcular_metricas = pd.DataFrame(data=y_pred,index=id_teste)

In [22]:
labels = pd.read_csv("material_trabalho/vendas_teste.csv")

score = r2_score(labels['Sales'],calcular_metricas)
rmse = math.sqrt(mean_squared_error(labels['Sales'],calcular_metricas))
mase = mean_absolute_error(labels['Sales'],calcular_metricas)
mae = median_absolute_error(labels['Sales'],calcular_metricas)
print('Score = ' + str(score))
#print('RMSE = ' + str(rmse))
print('MASE = ' + str(mase))
print('MAE = ' + str(mae))
print('RMSE = ' + str(rmse))

Score = 0.8969837154213123
MASE = 780.5805921529229
MAE = 507.58935546875
RMSE = 1232.5528005959713


In [ ]:
a
#Salvando modelo treinado
import pickle

with open('XGBoost.pickle', 'wb') as f:
    pickle.dump([modelo, dataset_treino, dataset_teste], f)

In [ ]:
a
import pickle
#Load modelo treinado
with open('XGBoost.pickle', 'rb') as f:
    modelo, dataset_treino, dataset_teste = pickle.load(f)